---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [11]:
'''G=P1_Graphs[4]

degrees=G.degree()
degree_values = sorted(set(degrees.values()))
histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]

print(histogram)'''

[0.136, 0.26, 0.2906666666666667, 0.168, 0.09866666666666667, 0.02666666666666667, 0.016, 0.0026666666666666666, 0.0013333333333333333]


In [12]:
'''
G=P1_Graphs[4]
clustering = nx.average_clustering(G)
shortest_path = nx.average_shortest_path_length(G)

print(clustering, shortest_path)'''

0.0033037037037037037 5.0785509568313305


In [ ]:
def graph_identification():
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [13]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [14]:
G.nodes(data=True)

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0}),
 (10, {'Department': 9, 'ManagementSalary': 0.0}),
 (11, {'Department': 14, 'ManagementSalary': 0.0}),
 (12, {'Department': 14, 'ManagementSalary': 1.0}),
 (13, {'Department': 26, 'ManagementSalary': 1.0}),
 (14, {'Department': 4, 'ManagementSalary': nan}),
 (15, {'Department': 17, 'ManagementSalary': 0.0}),
 (16, {'Department': 34, 'ManagementSalary': 0.0}),
 (17, {'Department': 1, 'ManagementSalary': 0.0}),
 (18, {'Department': 1, 'ManagementSalary': nan}),
 (19, {'Department': 14, 'Ma

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [21]:
def salary_predictions():
    from sklearn.ensemble import GradientBoostingClassifier
    
    df = pd.DataFrame(index=G.nodes())

    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    
    X_train = df[~df['ManagementSalary'].isnull()][['clustering','degree','degree_centrality','closeness','betweeness','pr']]
    y_train = df[~df['ManagementSalary'].isnull()]['ManagementSalary']
    
    X_test = df[df['ManagementSalary'].isnull()][['clustering','degree','degree_centrality','closeness','betweeness','pr']]
    
    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    pre=clf.predict_proba(X_test)[:,1]
    ret = pd.Series(pre, index=X_test.index)
    return ret

salary_predictions() #AUC: 0.95

252


1       0.008753
2       0.988256
5       0.988256
8       0.108871
14      0.044559
18      0.027113
27      0.035471
30      0.861790
31      0.310527
34      0.016056
37      0.044041
40      0.012103
45      0.007576
54      0.211637
55      0.599231
60      0.044722
62      0.988256
65      0.988256
77      0.013760
79      0.031205
97      0.009019
101     0.003401
103     0.670237
108     0.010602
113     0.035251
122     0.002173
141     0.076189
142     0.988256
144     0.009019
145     0.915127
          ...   
913     0.007438
914     0.008477
915     0.000759
918     0.030229
923     0.012984
926     0.039048
931     0.005443
934     0.000723
939     0.000723
944     0.000759
945     0.012984
947     0.025263
950     0.012475
951     0.004048
953     0.002195
959     0.000759
962     0.000723
963     0.034729
968     0.025263
969     0.034618
974     0.024442
984     0.000759
987     0.024442
989     0.025263
991     0.025263
992     0.000723
994     0.000759
996     0.0007

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [22]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [25]:
def new_connections_predictions():
    
    from sklearn.ensemble import GradientBoostingClassifier
    
    future_connections['pref_attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['common_neighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    
    X_train = future_connections[~future_connections['Future Connection'].isnull()][['pref_attachment','common_neighbors']]
    y_train = future_connections[~future_connections['Future Connection'].isnull()]['Future Connection']
    
    X_test = future_connections[future_connections['Future Connection'].isnull()][['pref_attachment','common_neighbors']]

    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    pre=clf.predict_proba(X_test)[:,1]
    ret = pd.Series(pre, index=X_test.index)

    return ret

new_connections_predictions() #AUC: 0.91

(107, 348)    0.031823
(542, 751)    0.012931
(20, 426)     0.543026
(50, 989)     0.013104
(942, 986)    0.013103
(324, 857)    0.013104
(13, 710)     0.148865
(19, 271)     0.141496
(319, 878)    0.013104
(659, 707)    0.012887
(49, 843)     0.013104
(208, 893)    0.012887
(377, 469)    0.012221
(405, 999)    0.022630
(129, 740)    0.017730
(292, 618)    0.017926
(239, 689)    0.013104
(359, 373)    0.012696
(53, 523)     0.032456
(276, 984)    0.013104
(202, 997)    0.013104
(604, 619)    0.119968
(270, 911)    0.013104
(261, 481)    0.069337
(200, 450)    0.825813
(213, 634)    0.012931
(644, 735)    0.088258
(346, 553)    0.012970
(521, 738)    0.012503
(422, 953)    0.017730
                ...   
(672, 848)    0.013104
(28, 127)     0.979748
(202, 661)    0.012998
(54, 195)     0.998514
(295, 864)    0.013104
(814, 936)    0.012931
(839, 874)    0.013103
(139, 843)    0.013104
(461, 544)    0.013183
(68, 487)     0.013183
(622, 932)    0.012887
(504, 936)    0.017232
(479, 528) 